In [8]:
from dotenv import load_dotenv
from groq import Groq
from IPython.display import Markdown, display

load_dotenv("../.env")

client = Groq()

In [9]:
completion = client.chat.completions.create(
    messages=[
        {"role":"user", 
         "content":"When I ask you question, I set temprature=0.7. But honestly I have no idea what is difference between temprature=0.7 and temprature=1.0. Can you explain it to me?"}
    ],
    model="llama-3.1-70b-versatile",
    temperature=0.7
)
response = completion.choices[0].message.content
display(Markdown(response))

In the context of a conversational AI model like me, `temperature` is a parameter that controls the randomness and creativity of the generated responses.

**Temperature (τ)** is a value between 0 and 1 that determines how "cautious" or "adventurous" the model is when generating text. Here's a rough interpretation of different temperature values:

* **Temperature = 0**: The model will generate the most likely response, based on the patterns it has learned from the training data. This will result in a very predictable and "safe" answer.
* **Temperature = 1**: The model will generate a more random and creative response, as it will explore more possibilities and take more risks. This can lead to more interesting and varied answers, but also increases the chance of errors or nonsensical responses.
* **Temperature between 0 and 1**: This is where things get interesting. A temperature value between 0 and 1 will balance the trade-off between predictability and creativity. A lower temperature (e.g., 0.3) will produce more conservative responses, while a higher temperature (e.g., 0.8) will produce more innovative ones.

In your case, setting `temperature=0.7` suggests that you want the model to generate responses that are somewhat creative and varied, but still generally reliable and accurate.

To give you a better sense of the difference, here are some possible characteristics of responses generated at different temperature values:

* `Temperature=0.7` (your setting): Responses are likely to be informative, relevant, and engaging, with some level of creativity and variation.
* `Temperature=1.0`: Responses may be more innovative, humorous, or thought-provoking, but also riskier and potentially less accurate.

Keep in mind that the ideal temperature value depends on the specific task, the model's quality, and your personal preferences. Experimenting with different temperature values can help you find the sweet spot for your needs.

In [10]:
class Agent:
    def __init__(self, system, client=client):
        self.system = system
        self.client = client
        self.messages = []

        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})
    
    def __call__(self, message):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content":result})
        return result
    
    def execute(self):
        completion = self.client.chat.completions.create(
            messages=self.messages,
            model="llama-3.1-70b-versatile",
            temperature=0.7
        )
        return completion.choices[0].message.content

In [11]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

def calculate(operation: str) -> float:
    return eval(operation)


def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0